In [30]:
import pandas as pd
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, plot_confusion_matrix
import seaborn as sns

In [31]:
data = pd.read_csv("wine_quality.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         6497 non-null   float64
 1   volatile.acidity      6497 non-null   float64
 2   citric.acid           6497 non-null   float64
 3   residual.sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free.sulfur.dioxide   6497 non-null   float64
 6   total.sulfur.dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB


## Divisão de base de dados e separação de base

In [39]:
#attributes = data[["fixed.acidity","volatile.acidity","citric.acid","residual.sugar","chlorides","free.sulfur.dioxide","total.sulfur.dioxide","density","pH","sulphates","alcohol"]]
#quality = data[["quality"]]

In [49]:
attributes = data.iloc[:, 0:11].values
quality = data.iloc[:, 11].values

X_train, X_test, y_train, y_test =  train_test_split(attributes, quality)

In [51]:
classifier = Sequential()
classifier.add(Dense(units=8, activation="linear",input_dim=11))
classifier.add(Dropout(0.1))
classifier.add(Dense(units=8, activation="linear"))
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1, activation="softmax"))
classifier.compile(optimizer="adam", loss="categorical_crossentropy",
                      metrics=["categorical_accuracy"])
history = classifier.fit(X_train, y_train, batch_size=16,
                  epochs=50,validation_split=0.1)

Epoch 1/50
274/274 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 2/50
274/274 [==============================] - 0s 957us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 3/50
274/274 [==============================] - 0s 988us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 4/50
274/274 [==============================] - 0s 873us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 5/50
274/274 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 6/50
274/274 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_lo

274/274 [==============================] - 0s 930us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 48/50
274/274 [==============================] - 0s 873us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 49/50
274/274 [==============================] - 0s 836us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
Epoch 50/50
274/274 [==============================] - 0s 880us/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_categorical_accuracy: 1.0000
